# map2loop: Process sills

Try to deal with sills by finding two longest contacts and defining new strat unit based on upper lower contact names. Need to clip by faults first. If sills intrude within unit can just use that name, but if there are two or more sills within a single unit need to even know this is happening. Also need to deal with diffeernt sills in contact with each other. Eurgh. In any case current codes will choke on limited extent sills, which they all are.

In [6]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import numpy as np
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import os
import rasterio
from rasterio.transform import from_origin
from rasterio import features
%matplotlib inline
import sys, os
from map2loop import m2l_utils
import geopandas as gpd
import pandas as pd
from math import acos, sqrt, cos, sin, degrees, radians, fabs, atan2
from shapely.geometry import shape, Polygon, LineString, Point
from shapely.geometry import Polygon
from shapely.geometry import Point
print(os.getcwd())


C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2loop\notebooks


In [7]:
gcode='GROUP_'
dcode='DIP'
ddcode='DIP_DIR'
ccode='CODE'
ncode='NAME'
descode='DESCRIPTN'
rcode='ROCKTYPE1'
ocode='OBJECTID'
mincode='MIN_AGE_MA'
maxcode='MAX_AGE_MA'

contact_decimate=5   # decimation of contact data (0 = no decimation)
mname='hams2'
test_data_path='../test_data/'
tmp_path=test_data_path+'tmp/'
data_path=test_data_path+'data/'
dtm_path=test_data_path+'dtm/'
output_path=test_data_path+'output/'

strat_graph_file=test_data_path+'graph/'+mname+'_strat.gml'

fault_file=data_path+mname+'_faults.shp'
structure_file=data_path+mname+'_structure.shp'
geology_file=data_path+mname+'_geol.shp'

dtm_file=dtm_path+mname+'_dtm.tif'
dtm_reproj_file=dtm_path+mname+'_dtm_rp.tif'


In [8]:
def get_groups(tmp_path,mname):
    f=open(tmp_path+mname+'_groups.txt',"r")
    groups =f.readlines()
    f.close

    ngroups=groups[0].split(" ")
    ngroups=int(ngroups[1])
    return(ngroups,groups)
    

In [9]:
geology = gpd.read_file(geology_file)
print(geology.crs)

geol_explode=m2l_utils.explode(geology)
print(geology.shape, geol_explode.shape)
roi = gpd.read_file(data_path+'hamms_roi.shp')

geol_clip=m2l_utils.clip_shp(geol_explode,roi)


dtm = rasterio.open(dtm_reproj_file)

ngroups,groups=get_groups(tmp_path,mname)

{'init': 'epsg:28350'}
(961, 63) (961, 63)


In [54]:
sill_ages=np.zeros((1000,4))
sill_names=np.zeros((1000),dtype='U25')
sills=0
sill_neighbour_info=np.zeros((1000,4)) #max age, min age, total length
sill_neighbour_names=np.zeros((1000),dtype='U25')

for i in range (0,ngroups):
    sill_neighbour_info[i,0]=-1e6 # neighbour max_age
    sill_neighbour_info[i,1]=1e6 # neighbour min_age
    sill_neighbour_info[i,2]=1e6 # neighbour length
    sill_neighbour_info[i,3]=i # neighbour index
    sill_neighbour_names[i]=''

summary=pd.read_csv(tmp_path+mname+'_all_sorts.txt')
summary.set_index("code", inplace=True)

In [72]:
j=0
allpts=0
ls_dict={}
ls_dict_decimate={}
id=0
for ageol in geol_clip.iterrows(): # central polygon
    ades=str(ageol[1][descode])
    arck=str(ageol[1][rcode])
    if(str(ageol[1][gcode])=='None'):
        agroup=str(ageol[1][ccode])
    else:
        agroup=str(ageol[1][gcode])
    
    for i in range(0,ngroups):
        if (gp_names[i]==agroup):
            if(int(ageol[1][maxcode]) > gp_ages[i][0]  ):
                gp_ages[i][0] = ageol[1][maxcode]
            if(int(ageol[1][mincode]) < gp_ages[i][1]  ):
                gp_ages[i][1] = ageol[1][mincode]

    if('intrusive' in arck and 'sill'  in ades):
        newgp=str(ageol[1][ccode])+'_'+str(ageol[1][ocode])
        #agp=str(ageol[1][gcode])
        print(newgp)
        if(str(ageol[1][gcode])=='None'):
            agp=str(ageol[1][ccode])
        else:
            agp=str(ageol[1][gcode])

        if(not newgp  in gp_names):
            gp_names[ngroups]=newgp
            gp_ages[ngroups][0]=ageol[1][maxcode]
            gp_ages[ngroups][1]=ageol[1][mincode]
            gp_ages[ngroups][2]=ngroups
            ngroups=ngroups+1

            
        neighbours=[]
        j+=1
        central_age=ageol[1][mincode]    #absolute age of central polygon
        central_poly=ageol[1].geometry
        for bgeol in geol_clip.iterrows(): #potential neighbouring polygons  
            if(ageol[1].geometry!=bgeol[1].geometry): #do not compare with self
                if (ageol[1].geometry.intersects(bgeol[1].geometry)): # is a neighbour
                    neighbours.append([(bgeol[1][ccode],bgeol[1][maxcode],bgeol[1][mincode],bgeol[1][rcode],bgeol[1][descode],bgeol[1].geometry)])  

        if(len(neighbours) ==1):
            continue            
            #print('this sill has 1 neighbour',len(neighbours))
        elif(len(neighbours) >1):
            #print('this sill has many neighbours',len(neighbours))
            for i in range(0,1000):
                sill_neighbour_info[i][0]=0
                sill_neighbour_info[i][1]=0
                sill_neighbour_info[i][2]=0
                sill_neighbour_names[i]=''
            n_sill_neighbours=0   
            for i in range (0,len(neighbours)):
                older_polygon=neighbours[i][0][5]
                if(not central_poly.is_valid ):
                    central_poly = central_poly.buffer(0)
                if(not older_polygon.is_valid):
                    older_polygon = older_polygon.buffer(0)
                LineStringC = central_poly.intersection(older_polygon)

                found=0
                for j in range(0,n_sill_neighbours):
                    if(neighbours[i][0][0]==sill_neighbour_names[j]):
                        sill_neighbour_info[j][2]=sill_neighbour_info[j][2]+LineStringC.length
                        found=True
                        break
                        
                if(not found):
                    sill_neighbour_info[n_sill_neighbours][0]=neighbours[i][0][1]                    
                    sill_neighbour_info[n_sill_neighbours][1]=neighbours[i][0][2]                    
                    sill_neighbour_info[n_sill_neighbours][2]=LineStringC.length
                    sill_neighbour_names[n_sill_neighbours]=neighbours[i][0][0]
                    n_sill_neighbours=n_sill_neighbours+1
                                
            sl=sill_neighbour_info[:n_sill_neighbours]
            ln=sl[:,2].argsort() #sort on lengths
            
            print('BEST',sill_neighbour_names[ln[n_sill_neighbours-1]],sill_neighbour_info[ln[n_sill_neighbours-1]][2])
            print('NEXT',sill_neighbour_names[ln[n_sill_neighbours-2]],sill_neighbour_info[ln[n_sill_neighbours-2]][2])
        else:
            continue
            #print('this sill has NO neighbours',len(neighbours))

            

A-FO-od_346
BEST A-FO-xo-a 26293.833048986307
NEXT A-FOp-bs 13711.303353187424
A-FO-od_565
A-FO-xo-a_513
BEST A-FOu-bbo 16342.527538505661
NEXT A-FOp-bs 15896.37081689976
A-FO-od_228
BEST A-FOu-bbo 6778.444335051038
NEXT A-FO-xo-a 5738.771242449317
A-FO-od_506
BEST A-FO-xo-a 14070.511078406584
NEXT A-FOp-bs 10335.450577540116
A-FO-od_190
A-FO-od_283
BEST A-FOh-xs-f 31015.626649643753
NEXT A-FOo-bbo 0.0
A-FO-od_700
BEST A-FOo-bbo 30440.09995714353
NEXT A-FOh-xs-f 28412.089834370934
A-FO-od_100
BEST A-FOj-xs-b 104527.40733605949
NEXT A-FOj-xs-b 104527.40733605949
A-FO-xo-a_644
A-FO-xo-a_114
BEST A-FOp-bs 14073.78657502074
NEXT A-FOu-bbo 10177.117479222974
A-FO-od_39
A-FO-xo-a_250
BEST A-FO-od 8348.005435038049
NEXT A-FOp-bs 7996.116620103851
A-FO-od_673
BEST A-FOj-xs-b 16171.84209781587
NEXT A-FOu-bbo 13416.031964727932
A-FO-xo-a_629
A-FO-od_640
BEST A-FOp-bs 104.26813754839314
NEXT A-FOu-bbo 75.50675169316649
A-FO-od_304
A-FO-od_257
BEST A-FOu-bbo 3094.8994745666005
NEXT A-FO-xo-a 2947.